In [1]:
from openmm.app import *
from openmm import *
from openmm.unit import *
import urllib.request
import sys

# ==========================================================
# Step 1: Download Protein Structure (Villin Headpiece)
# ==========================================================
# This solves the FileNotFoundError by fetching the structure directly from RCSB PDB.
pdb_id = '1UAO'
pdb_filename = 'Chignolin.pdb'

print(f"Downloading protein structure ({pdb_id})...")
urllib.request.urlretrieve(f'https://files.rcsb.org/download/{pdb_id}.pdb', pdb_filename)
pdb = PDBFile(pdb_filename)

# Using Amber14 force field and TIP3P water model
forcefield = ForceField('amber14-all.xml', 'amber14/tip3pfb.xml')

# ==========================================================
# Step 2: Set up the Simulation System
# ==========================================================
print("Creating the simulation system...")
# NonbondedMethod=NoCutoff is used for vacuum simulation (faster for testing)
system = forcefield.createSystem(pdb.topology, nonbondedMethod=NoCutoff, constraints=HBonds)

# Integrator settings: 300K temperature, 2fs time step
integrator = LangevinMiddleIntegrator(300*kelvin, 1/picosecond, 0.002*picoseconds)
simulation = Simulation(pdb.topology, system, integrator)
simulation.context.setPositions(pdb.positions)

# ==========================================================
# Step 3: Energy Minimization
# ==========================================================
print("Performing energy minimization...")
simulation.minimizeEnergy()

# ==========================================================
# Step 4: Run 100 ps Production Simulation
# ==========================================================
# 100 ps / 0.002 ps = 50,000 steps
total_steps = 50000 
record_interval = 1000 # Save a frame every 1000 steps

# Reporters: Output trajectory (PDB) and simulation data (CSV-like format to console)
simulation.reporters.append(PDBReporter('output_100ps.pdb', record_interval))
simulation.reporters.append(StateDataReporter(sys.stdout, record_interval, step=True, 
                             potentialEnergy=True, temperature=True, speed=True, progress=True, totalSteps=total_steps))

print(f"Starting 100 ps simulation ({total_steps} steps)...")
simulation.step(total_steps)

print("-" * 30)
print("✅ Task Completed Successfully!")
print(f"Files generated: {pdb_filename} (initial), output_100ps.pdb (trajectory)")

Creating the simulation system...
Performing energy minimization...
Starting 100 ps simulation (50000 steps)...
#"Progress (%)","Step","Potential Energy (kJ/mole)","Temperature (K)","Speed (ns/day)"
2.0%,1000,-230.44878005981445,293.59185747806384,0
4.0%,2000,-163.4290885925293,296.66125227549213,1.45e+03
6.0%,3000,-153.7827491760254,294.229084509821,1.43e+03
8.0%,4000,-131.9363136291504,275.20199961996843,1.41e+03
10.0%,5000,-195.6970977783203,263.49811771811176,1.42e+03
12.0%,6000,-207.39166259765625,321.67615710137073,1.41e+03
14.0%,7000,-266.84984016418457,313.4903422589181,1.42e+03
16.0%,8000,-206.16577911376953,271.6249541386929,1.42e+03
18.0%,9000,-172.58257865905762,282.20189095321683,1.42e+03
20.0%,10000,-220.14793014526367,307.7413094256685,1.41e+03
22.0%,11000,-149.214111328125,329.70833860804487,1.41e+03
24.0%,12000,-297.6597099304199,382.5636593127345,1.41e+03
26.0%,13000,-296.0078125,307.2956154481542,1.41e+03
28.0%,14000,-364.0967025756836,336.6233068864718,1.4e+03
30.0%